In [1]:
import os 
from pathlib import Path
import sys
import subprocess
import time
from typing import Optional
from collections import defaultdict 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from IPython.display import display
import latextable
from texttable import Texttable
from arguments_dict import arguments_dict
from collections import defaultdict 


executables = ['genmc', 'genmc-imm', 'genmc-wkmo', 'genmc-xmm']
executable_paths_dict = dict([(exe, Path("executables") / exe) for exe in executables])
test_names = ['LBn+ctrl', 'LBn+data', 'LBn', 'LBn-pairs']
tests = dict([(t, Path('tests') / t / 'variants' / (os.listdir(Path('tests') / t / 'variants')[0])) for t in test_names if os.path.isdir(Path('tests') / t)])
subprocess_timeout = 1000000 #s

class RunExecutableResult:
    class Timeout:
        pass
    class Ok:
        def __init__(self, output) -> None:
            super().__init__()
            self.output = output
    class Error:
        def __init__(self, retcode, output) -> None:
            super().__init__()
            self.retcode = retcode
            self.output = output

def run_executable(exe_path: Path, args: list[str], test_path: Path) -> RunExecutableResult:
    try:
        result = subprocess.run([exe_path, *args, test_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=subprocess_timeout)
        result_stdout = result.stdout.decode()
        result_stderr = result.stderr.decode()
        if result.returncode == 0:
            return RunExecutableResult.Ok(result_stdout + result_stderr)
        else:
            return RunExecutableResult.Error(result.returncode, result_stdout + result_stderr)
    except subprocess.TimeoutExpired:
        return RunExecutableResult.Timeout()

In [2]:
def get_execution_time(string: str) -> Optional[float]:
    try:
        pattern = "Total wall-clock time: "
        time = string.split(pattern)[1].split('s\n')[0]
        return float(time)
    except:
        return None
assert(get_execution_time(
"""
No errors were detected.
Number of complete executions explored: 3
Total wall-clock time: 0.05s
""") == 0.05)

def get_number_executions(string: str) -> Optional[int]:
    pattern = "Number of complete executions explored: "
    i = string.index(pattern) + len(pattern)
    number_str = ""
    while i < len(string) and string[i].isdigit():
        number_str += string[i]
        i += 1
    try:
        return int(number_str)
    except:
        return None

assert(get_number_executions("abc 123 Number of complete executions explored: 123456") == 123456)

def get_duplicate_executions(string: str) -> Optional[int]:
    try:
        pattern = "Number of duplicate executions explored: "
        i = string.index(pattern) + len(pattern)
        number_str = ""
        while i < len(string) and string[i].isdigit():
            number_str += string[i]
            i += 1
        return int(number_str)
    except:
        return None

assert(get_duplicate_executions("abc 123 Number of duplicate executions explored: 123456") == 123456)

def get_number_lb_races(string: str) -> Optional[int]:
    pattern = "Number of load buffering races explored: "
    try:
        i = string.index(pattern) + len(pattern)
        number_str = ""
        while i < len(string) and string[i].isdigit():
            number_str += string[i]
            i += 1
        return int(number_str)
    except:
        return None

assert(get_number_lb_races("abc 123 Number of load buffering races explored: 123456") == 123456)

class ResultItem:
    def __init__(self, execs, exe_time, dups, lb_races) -> None:
        self.execs = execs
        self.exe_time = exe_time
        self.dups = dups
        self.lb_races = lb_races

def run_and_get_results(exe_path: Path, args: list[str], test_path: Path) -> RunExecutableResult:
    res = run_executable(exe_path, args, test_path)
    if isinstance(res, RunExecutableResult.Ok):
        execs = get_number_executions(res.output)
        exe_time = get_execution_time(res.output)
        dups = get_duplicate_executions(res.output)
        lb_races = get_number_lb_races(res.output)
        if execs is None:
            # print(f"could not get number of executions in {res.output}")
            execs = 0
        if dups is None:
            dups = 0
        if exe_time is None:
            # print(f"could not get duplicate executions in {res.output}")
            exe_time = 0
        return RunExecutableResult.Ok(ResultItem(execs, exe_time, dups, lb_races))
    else:
        return res

timeout = '{\\fontspec{Symbola}\\symbol{"231B}}'

In [3]:
timeout = '{\\fontspec{Symbola}\\symbol{"231B}}'
rows = [["Test Name", "GenMC Execs", "IMM Execs", "WMC Execs", "WMC Dupls", "XMM Execs", "XMM Dupls", "LB Races"]]

times = defaultdict(lambda: [])
assert(executables == ['genmc', 'genmc-imm', 'genmc-wkmo', 'genmc-xmm'])
for test_name in test_names:
    for n in [10, 12, 14]:
        row = [f"{test_name}({n})"]
        for exe in executables:
            args = arguments_dict[exe] + ["--", f"-D N={n}"]
            res = run_and_get_results(executable_paths_dict[exe], args, tests[test_name])
            if isinstance(res, RunExecutableResult.Ok):
                execs = res.output.execs
                dups = res.output.dups
                exe_time = res.output.exe_time   
                times[f"{test_name}({n})"].append(f"{exe_time:.2f}s")
                lb_races = res.output.lb_races
                row += [execs, dups] if exe != "genmc" and exe != "genmc-imm" else [execs]
                if lb_races is not None:
                    row += [lb_races]
                    times[f"{test_name}({n})"].append(lb_races)
            elif isinstance(res, RunExecutableResult.Timeout):
                row += [timeout] * (2 if exe != "genmc" else 1)
            elif isinstance(res, RunExecutableResult.Error):
                print(f"Error: {res.output}")
                exit(1)
            else:
                assert(0)
        rows.append(row)

In [4]:
titles = ["Test Name", "GenMC$_\\RCMM$", "GenMC$_\\IMM$", "WMC", "GenMC-XMM", "WMC", "GenMC-XMM", "LB Races"] 
header = [[" ", "\\multicolumn{4}{c|}{Number of Executions}", "\\multicolumn{2}{c|}{Number of Duplicates}", "", "", "", "", ""], titles]

print(rows[-1])
table_rows = header + [[r[0], r[1], r[2], r[3], r[5], r[4], r[6], r[7]] for r in rows[1:]]

['LBn-pairs(14)', 2187, 16384, 16384, 61741, 16384, 48250, 57344]


In [5]:
def adjust_header(table: str) -> str:
    return table.replace("\\multicolumn{2}{c|}{Number of Duplicates} &  &  &  &  &", "\\multicolumn{2}{c|}{Number of Duplicates} &")

def adjust_width(table: str) -> str:
    return table.replace("\\begin{center}", "\\begin{center} \\resizebox{\\textwidth}{!}{").replace("\\end{tabular}", "\\end{tabular}}")

table_1 = Texttable()
table_1.set_cols_align(["l"] + ["r" for i in range(len(rows[0]) - 1)])
table_1.set_cols_valign(["m" for i in range(len(rows[0]))])
table_1.add_rows(table_rows)

table_1_code = latextable.draw_latex(table_1, position='H', caption='Load Buffering benchmarks run with the version of GenMC that WMC is based on ("Old GenMC$_\\RCMM$"), WMC, GenMC$_\\RCMM$, GenMC$_\\IMM$, and GenMC-XMM', label="table:lb-benchmarks")
table_1_code = adjust_header(adjust_width(table_1_code))

In [6]:
def adjust_header_time(table: str) -> str:
    return table.replace("\\multicolumn{5}{c|}{Execution Time} &  &  &  &", "\\multicolumn{5}{c|}{Execution Time}")

titles = ["Test Name", "GenMC$_\\RCMM$", "GenMC$_\\IMM$", "WMC", "GenMC-XMM", "LB Races"] 
header = [[" ", "\\multicolumn{5}{c|}{Execution Time}", "", "", "", ""], titles]

times_rows = header + [[test] + times[test] for test in times.keys()]

table_2 = Texttable()
table_2.set_cols_align(["l"] + ["r" for i in range(len(times_rows[0]) - 1)])
table_2.set_cols_valign(["m" for i in range(len(times_rows[0]))])
table_2.add_rows(times_rows)

table_2_code = latextable.draw_latex(table_2, position='H', caption='Load Buffering time benchmarks run with the version of GenMC that WMC is based on ("Old GenMC$_\\RCMM$"), WMC, GenMC$_\\RCMM$, GenMC$_\\IMM$, and GenMC-XMM', label="table:lb-time-benchmarks")
table_2_code = adjust_header_time(table_2_code)

print(table_1_code)
print(table_2_code)


\begin{table}[H]
	\begin{center} \resizebox{\textwidth}{!}{
		\begin{tabular}{|l|r|r|r|r|r|r|r|}
			\hline
			  & \multicolumn{4}{c|}{Number of Executions} & \multicolumn{2}{c|}{Number of Duplicates} &  \\
			\hline
			Test Name & GenMC$_\RCMM$ & GenMC$_\IMM$ & WMC & GenMC-XMM & WMC & GenMC-XMM & LB Races \\
			\hline
			LBn+ctrl(10) & 10 & 11 & 11 & 11 & 0 & 1 & 1 \\
			\hline
			LBn+ctrl(12) & 10 & 11 & 11 & 11 & 0 & 1 & 1 \\
			\hline
			LBn+ctrl(14) & 10 & 11 & 11 & 11 & 0 & 1 & 1 \\
			\hline
			LBn+data(10) & 1023 & 1024 & 1024 & 1024 & 0 & 1 & 10 \\
			\hline
			LBn+data(12) & 1023 & 1024 & 1024 & 1024 & 0 & 1 & 10 \\
			\hline
			LBn+data(14) & 1023 & 1024 & 1024 & 1024 & 0 & 1 & 10 \\
			\hline
			LBn(10) & 1023 & 1024 & 1024 & 1024 & 9 & 10 & 10 \\
			\hline
			LBn(12) & 4095 & 4096 & 4096 & 4096 & 11 & 12 & 12 \\
			\hline
			LBn(14) & 16383 & 16384 & 16384 & 16384 & 13 & 14 & 14 \\
			\hline
			LBn-pairs(10) & 243 & 1024 & 1024 & 1024 & 2101 & 2184 & 2560 \\
			\hline
			LB